Redbus Data Scraping with Selenium

In [ ]:
!pip install selenium
Downloads\PYTHON\RED BUS\Untitled Folder\streamlit_bus_filter_app.py

1.Government State Bus Transport: PUNJAB

In [ ]:
#PEPSU (Punjab) Bus Routes & Timings 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/pepsu/?utm_source=rtchometile"

def initialize_driver():
    """
    Initializes and returns a Chrome WebDriver instance.
    """
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    """
    Loads the specified URL and waits for the page to load.
    """
    driver.get(url)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "route"))
    )  # Wait until the routes are loaded
    time.sleep(2)  # Allow time for the page to load completely

def scrape_bus_routes(driver):
    """
    Scrapes the bus route links and names from the initial page.
    """
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

def scrape_bus_details(driver, url, route_name):
    """
    Scrapes bus details from a specific route page.
    """
    try:
        driver.get(url)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "button"))
        )  # Wait until the "View Buses" button is visible
        time.sleep(3)

        # Try to click the "View Buses" button if available
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            view_buses_button.click()
            time.sleep(5)  # Allow buses to load

            # Scroll down to load more content if needed
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for additional content to load

            # Collect bus details from the page
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to capture seat availability (different classes for different sections)
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

def scrape_all_pages():
    """
    Scrapes all pages of bus routes and details.
    """
    all_bus_details = []
    
    # Initialize WebDriver
    driver = initialize_driver()
    
    try:
        for page in range(1, 4):  # Scraping 3 pages of bus routes
            try:
                load_page(driver, URL)

                if page > 1:
                    # Adjust for page navigation
                    pagination_tab = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, f"//div[@class='DC_117_pageTabs '][text()='{page}']"))
                    )
                    driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                    driver.execute_script("arguments[0].click();", pagination_tab)
                    time.sleep(5)  # Wait for the page to load completely

                # Scrape bus routes on the current page
                all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)

                # Iterate through each route and scrape bus details
                for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                    bus_details = scrape_bus_details(driver, link, name)
                    if bus_details:
                        all_bus_details.extend(bus_details)

            except Exception as e:
                print(f"Error occurred while accessing page {page}: {str(e)}")
    finally:
        # Ensure the driver is properly quit even if an error occurs
        driver.quit()

    return all_bus_details

# Start the scraping process
all_bus_details = scrape_all_pages()

# Convert the list of bus details to a Pandas DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('punjab_bus_details.csv', index=False)

print("Scraping completed and data saved to 'punjab_bus_details.csv'.")


2.Government State Bus Transport: CHANDIGARH

In [ ]:
#Chandigarh Transport Undertaking (CTU) Bus Routes & Timings 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL for Chandigarh Transport Undertaking (CTU) buses
URL = "https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "route")))
    time.sleep(2)

def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "button")))
        time.sleep(3)

        try:
            view_buses_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "button")))
            view_buses_button.click()
            time.sleep(5)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)

            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error accessing {url}: {str(e)}")
        return []

def scrape_all_pages():
    all_bus_details = []
    driver = initialize_driver()
    
    try:
        for page in range(1, 4):
            try:
                load_page(driver, URL)

                if page > 1:
                    pagination_tab = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                    )
                    driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                    driver.execute_script("arguments[0].click();", pagination_tab)
                    time.sleep(5)

                all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)

                for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                    bus_details = scrape_bus_details(driver, link, name)
                    if bus_details:
                        all_bus_details.extend(bus_details)

            except Exception as e:
                print(f"Error on page {page}: {str(e)}")
    finally:
        driver.quit()

    return all_bus_details

all_bus_details = scrape_all_pages()
df = pd.DataFrame(all_bus_details)
df.to_csv('chandigarh_bus_details.csv', index=False)
print("Data saved to 'chandigarh_bus_details.csv'.")


3.Government State Bus Transport: KERALA

In [ ]:
#KSRTC (Kerala) Bus Routes & Timings
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website for Kerala KSRTC
URL = "https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile"

def initialize_driver():
    """
    Initializes and returns a Chrome WebDriver instance.
    """
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    """
    Loads the specified URL and waits for the page to load.
    """
    driver.get(url)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "route"))
    )  # Wait until the routes are loaded
    time.sleep(2)  # Allow time for page to load completely

def scrape_bus_routes(driver):
    """
    Scrapes the bus route links and names from the initial page.
    """
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

def scrape_bus_details(driver, url, route_name):
    """
    Scrapes bus details from a specific route page.
    """
    try:
        driver.get(url)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "button"))
        )  # Wait until the "View Buses" button is visible
        time.sleep(3)

        # Try to click the "View Buses" button if available
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            view_buses_button.click()
            time.sleep(5)  # Allow buses to load

            # Scroll down to load more content if needed
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for additional content to load

            # Collect bus details from the page
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to capture seat availability (different classes for different sections)
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

def scrape_all_pages():
    """
    Scrapes all pages of bus routes and details.
    """
    all_bus_details = []
    
    # Initialize WebDriver
    driver = initialize_driver()
    
    try:
        for page in range(1, 3):  # Scraping 2 pages of bus routes
            try:
                load_page(driver, URL)

                if page > 1:
                    # Updated pagination based on the page number HTML provided
                    pagination_tab = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs') and text()='{page}']"))
                    )
                    driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                    driver.execute_script("arguments[0].click();", pagination_tab)
                    time.sleep(5)  # Wait for the page to load completely

                # Scrape bus routes on the current page
                all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)

                # Iterate through each route and scrape bus details
                for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                    bus_details = scrape_bus_details(driver, link, name)
                    if bus_details:
                        all_bus_details.extend(bus_details)

            except Exception as e:
                print(f"Error occurred while accessing page {page}: {str(e)}")
    finally:
        # Ensure the driver is properly quit even if an error occurs
        driver.quit()

    return all_bus_details

# Start the scraping process
all_bus_details = scrape_all_pages()

# Convert the list of bus details to a Pandas DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('kerala_bus_details.csv', index=False)

print("Scraping completed and data saved to 'kerala_bus_details.csv'.")


4.Government State Bus Transport:RAJASTHAN   

In [ ]:
#KSRTC (Kerala) Bus Routes & Timings
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile"

def initialize_driver():
    """
    Initializes and returns a Chrome WebDriver instance.
    """
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    """
    Loads the specified URL and waits for the page to load.
    """
    driver.get(url)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "route"))
    )  # Wait until the routes are loaded
    time.sleep(2)  # Allow time for page to load completely

def scrape_bus_routes(driver):
    """
    Scrapes the bus route links and names from the initial page.
    """
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

def scrape_bus_details(driver, url, route_name):
    """
    Scrapes bus details from a specific route page.
    """
    try:
        driver.get(url)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "button"))
        )  # Wait until the "View Buses" button is visible
        time.sleep(3)

        # Try to click the "View Buses" button if available
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            view_buses_button.click()
            time.sleep(5)  # Allow buses to load

            # Scroll down to load more content if needed
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for additional content to load

            # Collect bus details from the page
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to capture seat availability (different classes for different sections)
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

def scrape_all_pages():
    """
    Scrapes all pages of bus routes and details.
    """
    all_bus_details = []
    
    # Initialize WebDriver
    driver = initialize_driver()
    
    try:
        for page in range(1, 3):  # Scraping 2 pages of bus routes
            try:
                load_page(driver, URL)

                if page > 1:
                    pagination_tab = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                    )
                    driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                    driver.execute_script("arguments[0].click();", pagination_tab)
                    time.sleep(5)  # Wait for the page to load completely

                # Scrape bus routes on the current page
                all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)

                # Iterate through each route and scrape bus details
                for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                    bus_details = scrape_bus_details(driver, link, name)
                    if bus_details:
                        all_bus_details.extend(bus_details)

            except Exception as e:
                print(f"Error occurred while accessing page {page}: {str(e)}")
    finally:
        # Ensure the driver is properly quit even if an error occurs
        driver.quit()

    return all_bus_details

# Start the scraping process
all_bus_details = scrape_all_pages()

# Convert the list of bus details to a Pandas DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('rajasthan_bus_details.csv', index=False)

print("Scraping completed and data saved to 'rajasthan_bus_details.csv'.")


5.Government State Bus Transport: TELANGANA

In [ ]:
#TGSRTC Bus Routes & Timings
import time
import pandas as pd
import random
from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# URL of the website
URL = "https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile"

def initialize_driver():
    """Initialize and return a Chrome WebDriver instance."""
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    """Load the given URL and wait for the page to load."""
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    time.sleep(random.uniform(2, 5))  # Random sleep time to simulate human interaction

def scrape_bus_routes(driver):
    """Scrape bus routes (links and names) from the main page."""
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

def scrape_bus_details(driver, url, route_name):
    """Scrape detailed information about buses for a given route."""
    try:
        driver.get(url)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "button")))

        # Click the "View Buses" button if available
        view_buses_button = driver.find_element(By.CLASS_NAME, "button")
        driver.execute_script("arguments[0].click();", view_buses_button)
        time.sleep(random.uniform(3, 5))

        # Scroll to the bottom to load all bus items
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(random.uniform(3, 5))

        # Extract bus details
        bus_details = []
        bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels")
        bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type")
        departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time")
        duration_elements = driver.find_elements(By.CLASS_NAME, "dur")
        reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time")
        star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        price_elements = driver.find_elements(By.CLASS_NAME, "fare")
        seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30')]")

        for i in range(len(bus_name_elements)):
            bus_detail = {
                "Route_Name": route_name,
                "Route_Link": url,
                "Bus_Name": bus_name_elements[i].text,
                "Bus_Type": bus_type_elements[i].text,
                "Departing_Time": departing_time_elements[i].text,
                "Duration": duration_elements[i].text,
                "Reaching_Time": reaching_time_elements[i].text,
                "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                "Price": price_elements[i].text,
                "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
            }
            bus_details.append(bus_detail)
        return bus_details

    except Exception as e:
        print(f"Error occurred while scraping bus details for {url}: {str(e)}")
        return []

def scrape_all_pages():
    """Scrape bus routes and details across multiple pages."""
    all_bus_details = []
    driver = initialize_driver()

    try:
        load_page(driver, URL)

        # Scrape bus route links and names
        all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)

        for link, name in zip(all_bus_routes_link, all_bus_routes_name):
            bus_details = scrape_bus_details(driver, link, name)
            if bus_details:
                all_bus_details.extend(bus_details)

    except Exception as e:
        print(f"Error occurred: {str(e)}")
    finally:
        driver.quit()

    return all_bus_details

# Scrape routes and details from all pages
all_bus_details = scrape_all_pages()

# Convert the list of bus details into a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('Telangana_bus_details.csv', index=False)

print("Bus details have been successfully saved to 'Telangana_bus_details.csv'.")



6.Government State Bus Transport: WEST BENGAL

In [ ]:
#West bengal transport corporation Bus Routes & Timings
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

# URL of the website
URL = "https://www.redbus.in/online-booking/west-bengal-transport-corporation?utm_source=rtchometile"

# Initialize WebDriver
driver = webdriver.Chrome()
driver.get(URL)
driver.maximize_window()

# Allow the page to load fully (wait 10 seconds to ensure all elements are loaded)
time.sleep(10)
print(f"Page successfully loaded: {URL}")

def scrape_bus_routes():
    """Scrape the list of bus routes from the main page."""
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Scrape the main page for route links and names
all_bus_routes_link, all_bus_routes_name = scrape_bus_routes()

def scrape_bus_details(url, route_name):
    """Scrape detailed information for each bus route."""
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Scroll down to load all bus items (wait for a few seconds)
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(3)  # Wait for more content to load
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # Find the details for each bus
        bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels")
        bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type")
        departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time")
        duration_elements = driver.find_elements(By.CLASS_NAME, "dur")
        reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time")
        star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
        price_elements = driver.find_elements(By.CLASS_NAME, "fare")
        seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

        # Collect bus details
        bus_details = []
        for i in range(len(bus_name_elements)):
            bus_detail = {
                "Route_Name": route_name,
                "Route_Link": url,
                "Bus_Name": bus_name_elements[i].text,
                "Bus_Type": bus_type_elements[i].text,
                "Departing_Time": departing_time_elements[i].text,
                "Duration": duration_elements[i].text,
                "Reaching_Time": reaching_time_elements[i].text,
                "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                "Price": price_elements[i].text,
                "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
            }
            bus_details.append(bus_detail)
        return bus_details

    except Exception as e:
        print(f"Error occurred while processing route: {url}. Error: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Scrape bus details for each route
for link, name in zip(all_bus_routes_link, all_bus_routes_name):
    bus_details = scrape_bus_details(link, name)
    if bus_details:
        all_bus_details.extend(bus_details)

# Convert the list of bus details into a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('WEST_BENGAL_bus_details.csv', index=False)
print("Bus details have been successfully saved to 'WEST_BENGAL_bus_details.csv'.")

# Close the driver
driver.quit()


7.Government State Bus Transport: ANDHRA PRADESH

In [ ]:
#APSRTC Bus Routes & Timings
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the APSRTC page
URL = "https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile"

def initialize_driver():
    """
    Initializes and returns a Chrome WebDriver instance.
    """
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    """
    Loads the specified URL and waits for the page to load.
    """
    driver.get(url)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "route"))
    )  # Wait until the routes are loaded
    time.sleep(2)  # Allow time for page to load completely

def scrape_bus_routes(driver):
    """
    Scrapes the bus route links and names from the initial page.
    """
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

def scrape_bus_details(driver, url, route_name):
    """
    Scrapes bus details from a specific route page.
    """
    try:
        driver.get(url)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "button"))
        )  # Wait until the "View Buses" button is visible
        time.sleep(3)

        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            view_buses_button.click()
            time.sleep(5)

            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)

            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details

        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

def scrape_all_pages():
    """
    Scrapes all pages of bus routes and details.
    """
    all_bus_details = []

    driver = initialize_driver()
    try:
        for page in range(1, 6):  # Scraping up to 5 pages of routes based on your input
            try:
                load_page(driver, URL)

                if page > 1:
                    # Locate the pagination tab using the class provided
                    pagination_tab = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, f"//div[@class='DC_117_pageTabs '][text()='{page}']"))
                    )
                    driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                    driver.execute_script("arguments[0].click();", pagination_tab)
                    time.sleep(5)

                all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)

                for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                    bus_details = scrape_bus_details(driver, link, name)
                    if bus_details:
                        all_bus_details.extend(bus_details)

            except Exception as e:
                print(f"Error occurred while accessing page {page}: {str(e)}")
    finally:
        driver.quit()

    return all_bus_details

# Start the scraping process
all_bus_details = scrape_all_pages()

# Convert the list of bus details to a Pandas DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('andhra_pradesh_bus_details.csv', index=False)

print("Scraping completed and data saved to 'andhra_pradesh_bus_details.csv'.")


8.Government State Bus Transport: UTTAR PRADESH

In [ ]:
#UPSRTC Bus Routes & Timings  
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the UPSRTC website
URL = "https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile"

def initialize_driver():
    """
    Initializes and returns a Chrome WebDriver instance.
    """
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    """
    Loads the specified URL and waits for the page to load.
    """
    driver.get(url)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "route"))
    )  # Wait until the routes are loaded
    time.sleep(2)  # Allow time for page to load completely

def scrape_bus_routes(driver):
    """
    Scrapes the bus route links and names from the initial page.
    """
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

def scrape_bus_details(driver, url, route_name):
    """
    Scrapes bus details from a specific route page.
    """
    try:
        driver.get(url)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "button"))
        )  # Wait until the "View Buses" button is visible
        time.sleep(3)

        # Try to click the "View Buses" button if available
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            view_buses_button.click()
            time.sleep(5)  # Allow buses to load

            # Scroll down to load more content if needed
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for additional content to load

            # Collect bus details from the page
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to capture seat availability (different classes for different sections)
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

def scrape_all_pages():
    """
    Scrapes all pages of bus routes and details.
    """
    all_bus_details = []
    
    # Initialize WebDriver
    driver = initialize_driver()
    
    try:
        for page in range(1, 6):  # Scraping 5 pages of bus routes
            try:
                load_page(driver, URL)

                if page > 1:
                    pagination_tab = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                    )
                    driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                    driver.execute_script("arguments[0].click();", pagination_tab)
                    time.sleep(5)  # Wait for the page to load completely

                # Scrape bus routes on the current page
                all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)

                # Iterate through each route and scrape bus details
                for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                    bus_details = scrape_bus_details(driver, link, name)
                    if bus_details:
                        all_bus_details.extend(bus_details)

            except Exception as e:
                print(f"Error occurred while accessing page {page}: {str(e)}")
    finally:
        # Ensure the driver is properly quit even if an error occurs
        driver.quit()

    return all_bus_details

# Start the scraping process
all_bus_details = scrape_all_pages()

# Convert the list of bus details to a Pandas DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('uttar_pradesh_bus_details.csv', index=False)

print("Scraping completed and data saved to 'uttar_pradesh_bus_details.csv'.")


9.Government State Bus Transport: SOUTH BENGAL

In [ ]:
#South Bengal State Transport Corporation (SBSTC) Bus Routes & Timings
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from selenium.common.exceptions import TimeoutException, WebDriverException

# URL for South Bengal State Transport Corporation (SBSTC)
URL = "https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile"

def initialize_driver():
    """
    Initializes and returns a Chrome WebDriver instance.
    """
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url, retries=3):
    """
    Loads the specified URL and waits for the page to load with retry mechanism.
    """
    attempts = 0
    while attempts < retries:
        try:
            driver.get(url)
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "route"))
            )  # Wait until the routes are loaded
            time.sleep(2)  # Allow time for page to load completely
            break
        except (TimeoutException, WebDriverException) as e:
            print(f"Error loading page, retrying... (Attempt {attempts + 1}/{retries})")
            attempts += 1
            if attempts == retries:
                raise Exception(f"Failed to load page {url} after {retries} attempts.")
            time.sleep(3)  # Wait a bit before retrying

def scrape_bus_routes(driver):
    """
    Scrapes the bus route links and names from the initial page.
    """
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

def scrape_bus_details(driver, url, route_name):
    """
    Scrapes bus details from a specific route page.
    """
    try:
        driver.get(url)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "button"))
        )  # Wait until the "View Buses" button is visible
        time.sleep(3)

        # Try to click the "View Buses" button if available
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            view_buses_button.click()
            time.sleep(5)  # Allow buses to load

            # Scroll down to load more content if needed
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for additional content to load

            # Collect bus details from the page
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to capture seat availability (different classes for different sections)
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

def scrape_all_pages():
    """
    Scrapes all pages of bus routes and details.
    """
    all_bus_details = []
    
    # Initialize WebDriver
    driver = initialize_driver()
    
    try:
        for page in range(1, 3):  # Scraping 2 pages of bus routes
            try:
                load_page(driver, URL)

                if page > 1:
                    pagination_tab = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                    )
                    driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                    driver.execute_script("arguments[0].click();", pagination_tab)
                    time.sleep(5)  # Wait for the page to load completely

                # Scrape bus routes on the current page
                all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)

                # Iterate through each route and scrape bus details
                for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                    bus_details = scrape_bus_details(driver, link, name)
                    if bus_details:
                        all_bus_details.extend(bus_details)

            except Exception as e:
                print(f"Error occurred while accessing page {page}: {str(e)}")
    finally:
        # Ensure the driver is properly quit even if an error occurs
        driver.quit()

    return all_bus_details

# Start the scraping process
all_bus_details = scrape_all_pages()

# Convert the list of bus details to a Pandas DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('south_bengal_bus_details.csv', index=False)

print("Scraping completed and data saved to 'south_bengal_bus_details.csv'.")


10.Government State Bus Transport: Himachal Pradesh

In [ ]:
#HRTC Bus Routes & Timings 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the HRTC (Himachal Pradesh) page
URL = "https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile"

def initialize_driver():
    """
    Initializes and returns a Chrome WebDriver instance.
    """
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    """
    Loads the specified URL and waits for the page to load.
    """
    driver.get(url)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "route"))
    )  # Wait until the routes are loaded
    time.sleep(2)  # Allow time for page to load completely

def scrape_bus_routes(driver):
    """
    Scrapes the bus route links and names from the initial page.
    """
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

def scrape_bus_details(driver, url, route_name):
    """
    Scrapes bus details from a specific route page.
    """
    try:
        driver.get(url)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "button"))
        )  # Wait until the "View Buses" button is visible
        time.sleep(3)

        # Try to click the "View Buses" button if available
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            view_buses_button.click()
            time.sleep(5)  # Allow buses to load

            # Scroll down to load more content if needed
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for additional content to load

            # Collect bus details from the page
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to capture seat availability (different classes for different sections)
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

def scrape_all_pages():
    """
    Scrapes all pages of bus routes and details.
    """
    all_bus_details = []
    
    # Initialize WebDriver
    driver = initialize_driver()
    
    try:
        for page in range(1, 5):  # Scraping 4 pages of bus routes
            try:
                load_page(driver, URL)

                if page > 1:
                    pagination_tab = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                    )
                    driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                    driver.execute_script("arguments[0].click();", pagination_tab)
                    time.sleep(5)  # Wait for the page to load completely

                # Scrape bus routes on the current page
                all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)

                # Iterate through each route and scrape bus details
                for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                    bus_details = scrape_bus_details(driver, link, name)
                    if bus_details:
                        all_bus_details.extend(bus_details)

            except Exception as e:
                print(f"Error occurred while accessing page {page}: {str(e)}")
    finally:
        # Ensure the driver is properly quit even if an error occurs
        driver.quit()

    return all_bus_details

# Start the scraping process
all_bus_details = scrape_all_pages()

# Convert the list of bus details to a Pandas DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('himachal_bus_details.csv', index=False)

print("Scraping completed and data saved to 'himachal_bus_details.csv'.") 
